In [ ]:
import os, csv, re

from matplotlib import pyplot as plt
import numpy as np
from sklearn.externals import joblib

# from audio_cnn_poutyne import NeuralNetwork, VGG_alt, OpenL3, TEST_LOADER, device
from train_coarse import VGG_alt, TEST_LOADER, device

from load_dataset import AudioDataset
import torch
from torch import nn

In [ ]:
checkpoint = torch.load("models/20190610_170308_coarse=0.787_fine=0.646.ckpt")

In [ ]:
## checkpoint must have been trained with same model architecture

# model = NeuralNetwork()
model = VGG_alt()
# model = OpenL3()

In [ ]:
model.load_state_dict(checkpoint)
model.eval()

In [ ]:
# # experiment; ignore for now

# class NN_Embeddings(nn.Module):
#     def __init__(self, original_model):
#         super(NN_Embeddings, self).__init__()
#         self.features = nn.Sequential(*list(original_model.children())[:-4])
        
#     def forward(self, x):
# #         print(x.shape)
#         x = self.features(x)
#         return x

In [ ]:
# # testing visualization; ignore for now

# emb = model_2(torch.from_numpy(np.zeros((1, 1, 128, 862)).astype(np.float32)))

# img_array = emb.detach().numpy()
# print(img_array.shape)
# print(np.sqrt(256))
# print(img_array[0].shape)
# img_array = img_array.reshape((32, 32))
# plt.imshow(img_array, cmap='hot', interpolation='nearest')

In [ ]:
# re-orders output as [fine, coarse] to match demo .csv file (maybe not necessary)
def reorder_labels_for_submission(list_or_array):
    if type(list_or_array) == torch.Tensor:
        list_or_array = list(list_or_array.to('cpu').numpy())
    list_or_array = list(list_or_array)
    return list_or_array[8:] + list_or_array[0:8]

In [ ]:
# process label names
label_names = joblib.load('label_order.pkl')
label_names = [re.sub('_presence', '', label_names[i]) for i in range(len(label_names))]

In [ ]:
# label_names

In [ ]:
FINE = False
COARSE = False
EMBED = True

with open('results_2019-5-6-aug.csv', 'w') as c:
    writer = csv.writer(c, delimiter=',')
    
    header = ['audio_filename'] + reorder_labels_for_submission(label_names)
    writer.writerow(header)
    basepath = r"D:\repos\Data-Processing\audio\validate_spec_vgg"
    data_rows = []
    for filename in os.listdir(basepath):
        audio_filename = filename[0:9] + '.wav'
        if EMBED:
            spectrogram, vgg, label = joblib.load(os.path.join(basepath, filename)) #emb
    #         print(spectrogram.shape, emb.shape, label.shape)
            spectrogram = np.expand_dims(spectrogram, axis=0)
            spectrogram = np.expand_dims(spectrogram, axis=0)
#             emb = emb.reshape((1, 256, 192))
#             emb = np.expand_dims(emb, axis=0)
#             emb = torch.from_numpy(emb)
    #         print(vgg.flatten().shape)
            vgg = torch.from_numpy(vgg.flatten().reshape((1, 1280)))
            spectrogram = spectrogram.astype(np.float32)
            spectrogram = torch.from_numpy(spectrogram)
    #         print(spectrogram.shape, emb.shape, label.shape)
    #         print(spectrogram.shape)
    #         spectrogram.to('cuda')
    #         print(vgg.shape, emb.shape)
            in_data = (spectrogram, vgg) #emb
        else:
            spectrogram, label = joblib.load(os.path.join(basepath, filename))
            spectrogram = np.expand_dims(spectrogram, axis=0)
            spectrogram = np.expand_dims(spectrogram, axis=0)
            spectrogram = spectrogram.astype(np.float32)
            spectrogram = torch.from_numpy(spectrogram)
            in_data = (spectrogram) #emb
        with torch.no_grad():
            results = model(in_data)
#             print(len(results[0]))
            results = torch.sigmoid(results[0])
            print(label)
            print(results)
            print()
#             img_array = emb.detach().numpy()
#             print()
            label_true = np.where(label == 1)[0]
            label_name = [label_names[i] for i in label_true]
#             print(label_name)
#             img_array = img_array.reshape((16, 16))
#             plt.imshow(img_array, cmap='hot', interpolation='nearest')
#             plt.show()
#             print(results.shape)
            results = results
            results = results.detach().numpy()
            if FINE:
                coarse_labels = []
                fine_label_names = label_names[8:]
                coarse_label_dict = {i: [] for i in range(8)}
                for i, r in enumerate(results):
#                     print(fine_label_names[i][0])
                    coarse_label_dict[int(fine_label_names[i][0])-1].append(r)
                for i in range(8):
                    coarse_labels.append(max(coarse_label_dict[i]))
#                 print('coarse labels', coarse_labels)
                results = coarse_labels + list(results)
            if COARSE:
                results = list(results) + [0 for i in range(29)]
            results = reorder_labels_for_submission(results)
            data_rows.append([audio_filename] + results)
#             print([audio_filename] + results)
    writer.writerows(data_rows)

In [ ]:
# run a command similar to this on the command line to get results
!python evaluate_predictions.py results_2019-5-6-aug.csv "D:\DCASE_2019\annotations.csv" "D:\DCASE_2019\dcase-ust-taxonomy.yaml"

In [ ]:
# python extract_embedding.py "D:\DCASE_2019\annotations.csv" $SONYC_UST_PATH/data $SONYC_UST_PATH/features $SONYC_UST_PATH/vggish